In [20]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map

In [37]:
sitename = "nzd0161"
transect_id = "nzd0161-0135"
water = pd.read_csv(f"../CoastSat/data/{sitename}/transect_time_series_tidally_corrected.csv")
water.index = pd.to_datetime(water.dates)
water = water[transect_id]
veg = pd.read_csv(f"Data/{sitename}/transect_time_series.csv")
veg.index = pd.to_datetime(veg.dates + "+00:00").dt.floor("s")
veg = veg[transect_id]
display((veg < water.mean()).value_counts())
veg = veg[veg < water.mean()]

nzd0161-0135
False    320
True      50
Name: count, dtype: int64

In [38]:
df = pd.DataFrame({"water": water, "veg": veg})
px.scatter(df, trendline="ols", title=transect_id)

In [39]:
from coastsat import SDS_transects

def despike(chainage, threshold=40):
    chainage = chainage.dropna()
    if len(chainage) < 5:
        return chainage
    chainage, dates = SDS_transects.identify_outliers(
        chainage.tolist(), chainage.index.tolist(), threshold
    )
    return pd.Series(chainage, index=dates)

despiked = despike(veg)
smoothed = despiked.rolling("180d", min_periods=1).mean()
df = pd.DataFrame({"water": water, "raw": veg, "despiked": despiked, "smoothed": smoothed})
px.scatter(df, trendline="ols", title=transect_id)

In [40]:
shorelines = gpd.read_file("../CoastSat/shorelines.geojson")
shorelines = shorelines[shorelines.id.str.startswith("nzd")]
sites = sorted(shorelines.id.unique())

In [71]:
water = pd.read_csv(f"../CoastSat/data/{sitename}/transect_time_series_tidally_corrected.csv").drop(columns="satname")
water.dates = pd.to_datetime(water.dates)
water.set_index("dates", inplace=True)
veg = pd.read_csv(f"Data/{sitename}/transect_time_series.csv").drop(columns="satname")
veg.dates = pd.to_datetime(veg.dates + "+00:00").dt.floor("s")
veg.set_index("dates", inplace=True)
veg = veg[veg.lt(water.rolling("30d", min_periods=1).mean())]

In [72]:
def process_site(sitename):
    water = pd.read_csv(f"../CoastSat/data/{sitename}/transect_time_series_tidally_corrected.csv").drop(columns="satname")
    water.dates = pd.to_datetime(water.dates)
    water.set_index("dates", inplace=True)

    try:
        veg = pd.read_csv(f"Data/{sitename}/transect_time_series.csv").drop(columns="satname")
    except:
        return
    veg.dates = pd.to_datetime(veg.dates + "+00:00").dt.floor("s")
    veg.set_index("dates", inplace=True)
    veg = veg[veg.lt(water.rolling("30d", min_periods=1).mean())]

    veg = veg.apply(despike, axis=0)

    for transect_id in veg.columns:
        veg[transect_id] = veg[transect_id].rolling("180d", min_periods=1).mean()

    veg.to_csv(f"Data/{sitename}/transect_time_series_smoothed.csv", index_label="dates")

process_map(
    process_site,
    shorelines.id.unique(),
    max_workers=32,
    chunksize=1,
    desc="Processing sites",
    position=0,
    leave=True,
)

*****

Processing sites:   0%|          | 0/562 [00:00<?, ?it/s]

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,